In [1]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob

OpenCV should be 4.8.0.76 Current version: 4.8.0


In [2]:
imgs_parent_path = "calibration_images/"
imgs_path = []
# Asumimos que las imágenes para la calibración se llaman: "1.jpg", "2.jpg", ...
for i in range(1,24):   # ajustar para calibrar
    imgs_path.append(imgs_parent_path + str(i) + ".jpg")

imgs = []
for path in imgs_path:
    imgs.append(cv2.imread(path))


In [3]:
good_imgs = []
all_corners = []
for img in imgs:
    corners = cv2.findChessboardCorners(img,(9,6))
    if corners[0]:
        all_corners.append(corners)
        good_imgs.append(img)
    print(corners[0])    # debug

corners = all_corners
imgs = good_imgs

True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [4]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
 
# To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) for image in imgs]
 
 
corners_refined = [cv2.cornerSubPix(i, cor[1], (9, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

In [5]:
corners_drawn_images = []
for i, cor in zip(imgs, corners_refined):
    corners_drawn_images.append(cv2.drawChessboardCorners(i, (9,6), cor, True))

In [6]:
def show_image(img):
    cv2.imshow("Imagen",img)
    cv2.waitKey()
    cv2.destroyAllWindows()
 
def write_image(name, img):
    cv2.imwrite(name, img)

imgs_path = [f"calibration_results/{i+1}.jpg" for i in range(len(imgs))]
for i in range(len(corners_drawn_images)):
    show_image(corners_drawn_images[i])
    write_image(imgs_path[i], corners_drawn_images[i])

In [7]:
def get_chessboard_points(chessboard_shape, dx, dy):
    puntos_real = []
    shape_x = chessboard_shape[0]
    shape_y = chessboard_shape[1]
    for x in range(shape_x):
        for y in range(shape_y):
            puntos_real.append(np.array([np.float32(x*dx),np.float32(y*dy), 0]))
    return np.array(puntos_real)

In [8]:
chessboard_points = [get_chessboard_points((9, 6), 30, 30) for _ in range(len(corners_refined))]
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
#valid_corners = [np.array(c,dtype=np.float32) for c in valid_corners]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)
#objp = get_chessboard_points((8, 6), 30, 30)  # (dx, dy) = 30 mm, por ejemplo
#chessboard_points = [objp.astype(np.float32) for _ in range(len(valid_corners))]
chessboard_points = np.asarray([get_chessboard_points((9, 6), 30, 30) for _ in range(len(valid_corners))],dtype = np.float32)

In [9]:
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(chessboard_points,valid_corners,imgs[0].shape[:2],None,None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[411.78611432   0.         359.48360445]
 [  0.         645.09326404 639.45371568]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[-9.26387620e-02  1.57344041e-03 -4.80084153e-03  2.67733470e-02
  -6.44198170e-06]]
Root mean squared reprojection error:
 206.78795933019396


In [10]:
def get_chessboard_points(chessboard_shape,
                          dx: float,
                          dy: float) -> np.ndarray:
    """
    Build the 3D object points for a planar chessboard lying on Z = 0.
    """
    cols, rows = chessboard_shape
    grid_xy = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
    objp = np.zeros((grid_xy.shape[0], 3), dtype=np.float32)
    objp[:, 0] = grid_xy[:, 0] * dx
    objp[:, 1] = grid_xy[:, 1] * dy
    return objp

objp_template = get_chessboard_points((9,6), 30, 30)
print("Número de puntos 3D del tablero:", objp_template.shape[0])

points_3d: List[np.ndarray] = []
points_2d: List[np.ndarray] = []

for refined in corners_refined:
    if isinstance(refined, np.ndarray) and refined.size > 0:
        points_3d.append(objp_template)
        points_2d.append(refined.astype(np.float32))

print(f"Número de frames utilizados para la calibración: {len(points_2d)}")

Número de puntos 3D del tablero: 54
Número de frames utilizados para la calibración: 22


In [11]:
image_size = imgs[0].shape[:2]
print("Tamaño de imagen:", image_size)

rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    points_3d,
    points_2d,
    image_size,
    None,
    None
)

print("Matriz intrínseca (cameraMatrix):\n", intrinsics)
print("Coeficientes de distorsión (distCoeffs):\n", dist_coeffs.ravel())
print("Error RMS de reproyección:", rms)

# no tiene sentido hacer la de extrínsecos para nuetsro caso de uso

import numpy as np

np.savez(
    "calibration_jesus.npz",
    cameraMatrix=intrinsics,
    distCoeffs=dist_coeffs
)


"""
BORRAR ESTE COMENTARIO LUEGO
CÓMO USAR LA CALIBRACIÓN:

...en el programa principal...
data = np.load("calibration.npz")
cameraMatrix = data["cameraMatrix"]
distCoeffs = data["distCoeffs"]

...en el loop de la cámara...
ret, frame = cap.read()

h, w = frame.shape[:2]

newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(
    cameraMatrix,
    distCoeffs,
    (w, h),
    1,
    (w, h)
)

undistorted = cv2.undistort(
    frame,
    cameraMatrix,
    distCoeffs,
    None,
    newCameraMatrix
)
"""

Tamaño de imagen: (720, 1280)
Matriz intrínseca (cameraMatrix):
 [[1.00134159e+03 0.00000000e+00 6.53584349e+02]
 [0.00000000e+00 9.98422162e+02 3.47420749e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Coeficientes de distorsión (distCoeffs):
 [-0.2418064   0.96576773 -0.00410288  0.01401859 -1.28004347]
Error RMS de reproyección: 1.726524483621173


'\nBORRAR ESTE COMENTARIO LUEGO\nCÓMO USAR LA CALIBRACIÓN:\n\n...en el programa principal...\ndata = np.load("calibration.npz")\ncameraMatrix = data["cameraMatrix"]\ndistCoeffs = data["distCoeffs"]\n\n...en el loop de la cámara...\nret, frame = cap.read()\n\nh, w = frame.shape[:2]\n\nnewCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(\n    cameraMatrix,\n    distCoeffs,\n    (w, h),\n    1,\n    (w, h)\n)\n\nundistorted = cv2.undistort(\n    frame,\n    cameraMatrix,\n    distCoeffs,\n    None,\n    newCameraMatrix\n)\n'